In [1]:
import os
import numpy as np
print("hello")
import pandas as pd
import datetime
import matplotlib.pyplot as plt

hello


ImportError: cannot import name '_imaging' from 'PIL' (D:\program\Python\Python37\Lib\site-packages\PIL\__init__.py)

In [5]:
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, RepeatVector, Lambda
import keras.backend as K
from sklearn.preprocessing import LabelEncoder

ImportError: cannot import name '_c_internal_utils' from partially initialized module 'matplotlib' (most likely due to a circular import) (D:\program\Python\Python37\Lib\site-packages\matplotlib\__init__.py)

In [ ]:
# Configurable parameters
DATA_FILE_PATH = '/Users/dharmik/Downloads/compressed-results-2023-05-05 00_00_00.parquet'
MODEL_SAVE_PATH = '/Users/dharmik/Desktop/Model1'
ENCODER_LOAD_PATH = '/Users/dharmik/Downloads/mac-RA/encoder.joblib'
BATCH_SIZE = 32
EPOCHS = 1
N_STEPS = 12

In [ ]:
# Dataset columns
REQUIRED_COLUMNS = ['min_osm_id', 'timestamp', 'speed', 'freeFlow']

In [ ]:
def check_nan_values(df):
    # drop rows with NaN values in the dataframe
    df = df.dropna()
    return df

In [ ]:
def convert_datetime(timestamp):
    try:
        return pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S')

def rec_data_prp(data):
    if not set(REQUIRED_COLUMNS).issubset(data.columns):
        raise ValueError(f"Dataset must contain the following columns: {REQUIRED_COLUMNS}")
    data = data.sort_values(by=['min_osm_id', 'timestamp'])
    data['timestamp'] = data['timestamp'].apply(convert_datetime)
    data['time'] = data['timestamp'].dt.strftime('%H')
    data['day'] = data['timestamp'].dt.weekday
    data['day'] = data['day'].apply(lambda x: '1' if x <= 4 else '0')  # weekday is 1 and weekend is 0
    data['target15'] = data.speed.shift(-3)
    data['target15'].fillna(data['freeFlow'], inplace=True)
    data['target30'] = data.speed.shift(-6)
    data['target30'].fillna(data['freeFlow'], inplace=True)
    data['target45'] = data.speed.shift(-9)
    data['target45'].fillna(data['freeFlow'], inplace=True)
    data['target60'] = data.speed.shift(-12)
    data['target60'].fillna(data['freeFlow'], inplace=True)
    return data

# Create a 3D sequence of data
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x = sequence[i:end_ix, :6]
        seq_y = sequence[end_ix, -4:]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def load_encoder_and_rmv_ovrlp(data, n_steps=N_STEPS):
    data = rec_data_prp(data)
    
    # Load the saved encoder
    label_encoder = joblib.load(ENCODER_LOAD_PATH)
    data['min_osm_id'] = label_encoder.transform(data['min_osm_id'].astype(str))
    
    # Convert the 'timestamp' column to Unix time
    data['timestamp'] = data['timestamp'].astype(np.int64) // 10**9
    
    data = data[['min_osm_id', 'timestamp', 'day', 'time', 'speed', 'freeFlow', 'target15', 'target30', 'target45', 'target60']]
    data = data.values
    a1, a2 = split_sequence(data, n_steps)
    X, y = [], []
    for i in range(len(a1)):
        X.append(a1[i])
        y.append(a2[i])
    
    # Convert X and y to float32 data type
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    return X, y

def speed_activation(x, freeFlow_speed):
    freeFlow_speed = K.expand_dims(freeFlow_speed, axis=1)
    return K.minimum(x, freeFlow_speed)

In [ ]:
# Configure the model
def model_config(X, y, batch_size, n_steps):
    X = X[:, :, 2:6]
    n_features = X.shape[2]
    model = Sequential()
    #model.add(TimeDistributed(Dense(5, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(TimeDistributed(Dense(5, activation='relu'), input_shape=(n_steps, X.shape[-1])))
    model.add(LSTM(5, activation='tanh', batch_input_shape=(batch_size, n_steps, n_features)))
    model.add(Dense(4))
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
    return model

# Fit the model
def model_fit(model, X, y, batch_size, epochs):
    X = X[:, :, 2:6]
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=False)
    train_X = np.asarray(train_X).astype('float32')
    test_X = np.asarray(test_X).astype('float32')
    train_y = np.asarray(train_y).astype('float32')
    test_y = np.asarray(test_y).astype('float32')
    for epoch in range(epochs):
        model.fit(train_X, train_y, epochs=1, batch_size=batch_size, validation_data=(test_X, test_y), shuffle=True)
        model.reset_states()
    return model

# make prediction
#print accuracy and show the prediction  
def forecast_lstm(model, data): 
    X, y = rmv_ovrlp(data)  
    X1 = X[:, :, 2:6]
    yhat = model.predict(X1)
    print('The prediction accuracy (MAPE) for next 15 minutes interval is: %.2f' % (MAPE(yhat[:,0], y[:,0])*100))
    bins = np.arange(-5, 5.5, 0.25) #range and size of bins
    # diff1 = yhat[:,0] - y[:,0]
    # plt.hist(diff1, bins = bins)
    # plt.ylabel('Frequency')
    # plt.title('Histogram of prediction errors for next 15 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    print('The prediction accuracy (MAPE) for next 30 minutes interval is: %.2f' % (MAPE(yhat[:,1], y[:,1])*100))
    # diff2 = yhat[:,1] - y[:,1]
    # plt.hist(diff2, bins = bins)
    # plt.title('Histogram of prediction errors for next 30 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    print('The prediction accuracy (MAPE) for next 45 minutes interval is: %.2f' % (MAPE(yhat[:,2], y[:,2])*100))
    # diff3 = yhat[:,2] - y[:,2]
    # plt.hist(diff3, bins = bins)
    # plt.title('Histogram of prediction errors for next 45 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    print('The prediction accuracy (MAPE) for next 60 minutes interval is: %.2f' % (MAPE(yhat[:,3], y[:,3])*100))
    # diff4 = yhat[:,3] - y[:,3]
    # plt.hist(diff4, bins = bins)
    # plt.title('Histogram of prediction errors for next 60 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    output = np.concatenate((X[:, 0, :], yhat), axis=1)
    output = pd.DataFrame(output, columns=['min_osm_id', 'timestamp', 'day', 'time', 'speed', 'freeFlow', 'Speed15', 'Speed30', 'Speed45', 'Speed60'])
    output = output.round({'speed': 1, 'freeFlow': 1, 'Speed15': 1, 'Speed30': 1, 'Speed45': 1, 'Speed60': 1})
    
    return output

# Evaluate model performance
def evaluate_model(model, X, y):
    X = X[:, :, 2:6]
    y_pred = model.predict(X)
    mape = MAPE(y, y_pred)
    print(f"Mean Absolute Percentage Error: {mape}")
    return mape

# Run a repeated experiment
def run(data_file_path, model_save_path, batch_size=BATCH_SIZE, epochs=EPOCHS, n_steps=N_STEPS):
    # Load dataset
    if not os.path.exists(data_file_path):
        raise ValueError(f"Dataset file not found at: {data_file_path}")
    df = pd.read_parquet(data_file_path).drop('Unnamed: 0', axis=1)
    df = check_nan_values(df)

    # Fit the base model
    X, y = load_encoder_and_rmv_ovrlp(df)
    model = model_config(X, y, batch_size, n_steps)
    lstm_model = model_fit(model, X, y, batch_size, epochs)

    # Evaluate the model
    evaluate_model(lstm_model, X, y)

    # Save the model
    if not os.path.exists(os.path.dirname(model_save_path)):
        os.makedirs(os.path.dirname(model_save_path))
    lstm_model.save(model_save_path)
    return lstm_model

# Entry point
model = run(DATA_FILE_PATH, MODEL_SAVE_PATH, BATCH_SIZE, EPOCHS, N_STEPS)

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, RepeatVector, Lambda
import keras.backend as K
from sklearn.preprocessing import LabelEncoder
# Configurable parameters
#DATA_FILE_PATH = '/Users/dharmik/Downloads/mac-RA/merged_data.csv'
#MODEL_SAVE_PATH = '/Users/dharmik/Desktop/'
BATCH_SIZE = 32
EPOCHS = 1
N_STEPS = 12



# Dataset columns
REQUIRED_COLUMNS = ['min_osm_id', 'timestamp', 'speed', 'freeFlow']


def convert_datetime(timestamp):
    try:
        return pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S')
    
    
# Preprocess dataset
def rec_data_prp(data):
    if not set(REQUIRED_COLUMNS).issubset(data.columns):
        raise ValueError(f"Dataset must contain the following columns: {REQUIRED_COLUMNS}")
    data = data.sort_values(by=['min_osm_id', 'timestamp'])
    data['timestamp'] = data['timestamp'].apply(convert_datetime)
    data['time'] = data['timestamp'].dt.strftime('%H')
    data['day'] = data['timestamp'].dt.weekday
    data['day'] = data['day'].apply(lambda x: '1' if x <= 4 else '0')  # weekday is 1 and weekend is 0
    data['target15'] = data.speed.shift(-3)
    data['target15'].fillna(data['freeFlow'], inplace=True)
    data['target30'] = data.speed.shift(-6)
    data['target30'].fillna(data['freeFlow'], inplace=True)
    data['target45'] = data.speed.shift(-9)
    data['target45'].fillna(data['freeFlow'], inplace=True)
    data['target60'] = data.speed.shift(-12)
    data['target60'].fillna(data['freeFlow'], inplace=True)
    return data

def check_nan_values(df):
    # drop rows with NaN values in the dataframe
    df = df.dropna()
    return df


# Create a 3D sequence of data
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x = sequence[i:end_ix, :6]
        seq_y = sequence[end_ix, -4:]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def load_encoder_and_rmv_ovrlp(data, n_steps=N_STEPS):
    data = rec_data_prp(data)
    
    # Load the saved encoder
    label_encoder = joblib.load(ENCODER_LOAD_PATH)
    data['min_osm_id'] = label_encoder.transform(data['min_osm_id'].astype(str))
    
    # Convert the 'timestamp' column to Unix time
    data['timestamp'] = data['timestamp'].astype(np.int64) // 10**9
    
    data = data[['min_osm_id', 'timestamp', 'day', 'time', 'speed', 'freeFlow', 'target15', 'target30', 'target45', 'target60']]
    data = data.values
    a1, a2 = split_sequence(data, n_steps)
    X, y = [], []
    for i in range(len(a1)):
        X.append(a1[i])
        y.append(a2[i])
    
    # Convert X and y to float32 data type
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    return X, y

def speed_activation(x, freeFlow_speed):
    freeFlow_speed = K.expand_dims(freeFlow_speed, axis=1)
    return K.minimum(x, freeFlow_speed)

# # Configure the model
# def model_config(X, y, batch_size, n_steps):
#     X = X[:, :, 2:6]
#     n_features = X.shape[2]
#     model = Sequential()
#     #model.add(TimeDistributed(Dense(5, activation='relu'), input_shape=(n_steps, n_features)))
#     model.add(TimeDistributed(Dense(5, activation='relu'), input_shape=(n_steps, X.shape[-1])))
#     model.add(LSTM(5, activation='tanh', batch_input_shape=(batch_size, n_steps, n_features)))
#     model.add(Dense(4))
#     model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
#     return model

# # Fit the model
# def model_fit(model, X, y, batch_size, epochs):
#     X = X[:, :, 2:6]
#     train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=False)
#     train_X = np.asarray(train_X).astype('float32')
#     test_X = np.asarray(test_X).astype('float32')
#     train_y = np.asarray(train_y).astype('float32')
#     test_y = np.asarray(test_y).astype('float32')
#     for epoch in range(epochs):
#         model.fit(train_X, train_y, epochs=1, batch_size=batch_size, validation_data=(test_X, test_y), shuffle=True)
#         model.reset_states()
#     return model

# Configure the model
def model_config(X, y):
    X = X[:, 0, 2:6]
    n_features = X.shape[1]
    model = Sequential()
    model.add(Dense(5, activation='relu', batch_input_shape=(1, 1, n_features)))
    model.add(LSTM(5, activation='tanh', stateful=True))
    model.add(Dense(4))
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
    return model

# Fit the model
def model_fit(model, X, y, epochs):
    X = X[:, 0, 2:6]
    n_features = X.shape[1]
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=False)
    train_X = np.asarray(train_X).astype('float32')
    test_X = np.asarray(test_X).astype('float32')
    train_y = np.asarray(train_y).astype('float32')
    test_y = np.asarray(test_y).astype('float32')
    for epoch in range(epochs):
        for i in range(len(train_X)):
            model.train_on_batch(np.reshape(train_X[i], (1, 1, n_features)), np.reshape(train_y[i], (1, -1)))
        model.reset_states()
        for i in range(len(test_X)):
            model.test_on_batch(np.reshape(test_X[i], (1, 1, n_features)), np.reshape(test_y[i], (1, -1)))
    return model


# make prediction
#print accuracy and show the prediction  
def forecast_lstm(model, data):
    X, y = load_encoder_and_rmv_ovrlp(data)
    X1 = X[:, :, 2:6] # select relevant features
    n_features = X1.shape[2]
    n_steps = X1.shape[1]
    yhat = np.empty_like(y)
    for i in range(len(X1)):
        yhat[i] = model.predict(np.reshape(X1[i], (1, n_steps, n_features)))
    print('The prediction accuracy (MAPE) for next 15 minutes interval is: %.2f' % (MAPE(yhat[:,0], y[:,0])*100))
    bins = np.arange(-5, 5.5, 0.25) #range and size of bins
    plt.hist(yhat[:,0] - y[:,0], bins, alpha=0.5, color='r', label='forecast') #forecast histogram
    plt.hist(X[:,11,3] - y[:,0], bins, alpha=0.5, color='b', label='persistence') #persistence histogram
    plt.legend(loc='upper right')
    plt.show()

    # diff1 = yhat[:,0] - y[:,0]
    # plt.hist(diff1, bins = bins)
    # plt.ylabel('Frequency')
    # plt.title('Histogram of prediction errors for next 15 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    print('The prediction accuracy (MAPE) for next 30 minutes interval is: %.2f' % (MAPE(yhat[:,1], y[:,1])*100))
    # diff2 = yhat[:,1] - y[:,1]
    # plt.hist(diff2, bins = bins)
    # plt.title('Histogram of prediction errors for next 30 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    print('The prediction accuracy (MAPE) for next 45 minutes interval is: %.2f' % (MAPE(yhat[:,2], y[:,2])*100))
    # diff3 = yhat[:,2] - y[:,2]
    # plt.hist(diff3, bins = bins)
    # plt.title('Histogram of prediction errors for next 45 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    print('The prediction accuracy (MAPE) for next 60 minutes interval is: %.2f' % (MAPE(yhat[:,3], y[:,3])*100))
    # diff4 = yhat[:,3] - y[:,3]
    # plt.hist(diff4, bins = bins)
    # plt.title('Histogram of prediction errors for next 60 minutes')
    # plt.xlabel('Relative percentage error')
    # plt.ylabel('Frequency')
    # plt.gcf().set_dpi(300)
    # plt.show()
    #..........
    output = np.concatenate((X[:, 0, :], yhat), axis=1)
    output = pd.DataFrame(output, columns=['min_osm_id', 'timestamp', 'day', 'time', 'speed', 'freeFlow', 'Speed15', 'Speed30', 'Speed45', 'Speed60'])
    output = output.round({'speed': 1, 'freeFlow': 1, 'Speed15': 1, 'Speed30': 1, 'Speed45': 1, 'Speed60': 1})
    
    return output

# Evaluate model performance
# def evaluate_model(model, X, y):
#     X = X[:, :, 2:6]
#     y_pred = model.predict(X)
#     mape = MAPE(y, y_pred)
#     print(f"Mean Absolute Percentage Error: {mape}")
#     return mape

def evaluate_model(model, X, y):
    X = X[:, :, 2:6]  # take only 2nd to 5th features
    n_features = X.shape[2]  # number of features
    n_steps = X.shape[1]  # number of time steps
    y_pred = np.empty_like(y)
    for i in range(len(X)):
        X_sample = np.reshape(X[i], (1, n_steps, n_features))
        y_pred[i] = model.predict(X_sample)
    mape = MAPE(y, y_pred)
    print(f"Mean Absolute Percentage Error: {mape}")
    return mape

    
model = tf.keras.models.load_model('/Users/dharmik/Desktop/Model1/', compile=False)
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

df = pd.read_parquet('/Users/dharmik/Downloads/compressed-results-2023-05-05 00_00_00.parquet').drop('Unnamed: 0', axis = 1)
df = check_nan_values(df)


# df = df.iloc[99000000:99700000,:] #selecting Tuesday October 27th 6:

df = df.sort_values(by=['min_osm_id', 'timestamp'])
output1 = forecast_lstm(model, df)